In [1]:
!pip install pandas
!pip install beautifulsoup4
!pip install twilio

from twilio.rest import Client
import pandas as pd
from datetime import datetime
import numpy as np
import bs4 as bs
import requests

     |████████████████████████████████| 419kB 7.7MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/e6/14/12/e54011f7e5c75ca5d4ed4d275f5300837f9a733ed306259621
Successfully built twilio


In [11]:
def scrape_wiki():
    """Scrapes wikipedia information using Beautiful Soup. Returns the scheduled events table in html"""
    wiki_link = 'https://en.wikipedia.org'
    weblink = wiki_link + '/wiki/List_of_UFC_events'
    response = requests.get(weblink)
    soup = bs.BeautifulSoup(response.text, 'lxml')
    scheduled_events = soup.find_all('table', {'class': 'wikitable'})[0]
    return scheduled_events

In [29]:
def create_event_df(table):
    """Converts html sceduled events table into a dataframe"""
    ufc_events = pd.read_html(str(table), header=0)[0]
    ufc_events.drop(columns='Ref.', inplace=True)
    ufc_events.rename(columns={'Unnamed: 2': 'Venue', 'Original Date': 'Date'}, inplace=True)
    ufc_events['Days til event'] = None
    # reformat hyperlinks if available
    links = []
    for row in table.find_all('tr')[1:]:
        link = row.find_all('td')[0].contents[0]
        try:
            links.append(wiki_link + link.get('href'))
        except:
            links.append("no link")
    ufc_events['Fight Card Link'] = links
    # fix date format
    days = []
    for index, row in ufc_events.iterrows():
        old_date = row['Date']
        new_date = datetime.strptime(old_date, '%b %d, %Y')
        row['Days til event'] = (new_date-datetime.now()).days
        row['Date'] = new_date.strftime('%d %b %Y')
    return ufc_events

In [30]:
scheduled_events = scrape_wiki()
ufc_events = create_event_df(scheduled_events)
ufc_events.tail(2)

,Event,Date,Venue,Location,Notes,Days til event,Fight Card Link
8,UFC on ESPN: Smith vs. Teixeira,13 May 2020,VyStar Veterans Memorial Arena,"Jacksonville, Florida, U.S.",Planned,7,https://en.wikipedia.org/wiki/UFC_on_ESPN:_Smi...
9,UFC 249: Ferguson vs. Gaethje,09 May 2020,VyStar Veterans Memorial Arena,"Jacksonville, Florida, U.S.",Planned,3,https://en.wikipedia.org/wiki/UFC_249


In [32]:
def send_whatsapp(recipient, message_body):
    """Send next event info via Whatsapp using the Twilio API"""
    message = client.messages.create(body=message_body,
                                     from_='whatsapp:+',
                                     to=recipient)
    print(message.sid)

In [33]:
account_sid = '' # enter Twilio id
auth_token = '' # enter Twilio token
client = Client(account_sid, auth_token)
jay = 'whatsapp:+' # enter recipients phone number in format 'whatsapp:+############'

next_ufc_event = ufc_events[ufc_events['Days til event'] < 6]
if not next_ufc_event.shape[0] == 0:
    event_name_and_card = next_ufc_event['Event'] + " (" + next_ufc_event['Fight Card Link'] + ")"
    message_body = 'Your {} appointment is coming up on {}.'.format(event_name_and_card.values[0], next_ufc_event['Date'].values[0])
    print(message_body)
    send_whatsapp(jay, message_body)

Your UFC 249: Ferguson vs. Gaethje (https://en.wikipedia.org/wiki/UFC_249) appointment is coming up on 09 May 2020.
SM542669c5b8cb4f1993573147c70be4d8
